In [78]:
%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
from tqdm import tqdm
from os import getcwd, path as base_path
from spacy import displacy
from collections import Counter
import en_core_web_sm
from typing import List

import io

In [2]:
from sys import path
PROJECT_PATH = getcwd()[:getcwd().find("notebooks")][:-1]
path.append(f"{PROJECT_PATH}\\src")

In [79]:
from utils import find_urls_in_text

In [3]:
PROJECT_NAME = "wsj_tweet_scrapping"
PROJECT_PATH = getcwd()[: getcwd().find(PROJECT_NAME) + len(PROJECT_NAME)]
DATA_PATH = f"{PROJECT_PATH}\\data"
EXCEL_PATH = f"{DATA_PATH}\\excel"

In [4]:
from_year = 2011
to_year = 2020
news_df = pd.read_excel(f"{EXCEL_PATH}\\wsj_logistics_news_{from_year}-{to_year}.xlsx")
news_df.dropna(inplace=True)

In [5]:
news_df.head(6)

,data-item-id,data-conversation-id,date,tweet,url,text,clean_news_text
0,138783141312741408,138783141312741408,2011-11-21,Chinese Media Firm Hit by Muddy Waters on.ws...,http://on.wsj.com/vMZ1s2,Focus Media Holding Ltd. lashed back Tuesday a...,focus media holding ltd. lashed back tuesday m...
1,138950373275738096,138950373275738096,2011-11-22,China Pushes Clean-Energy Agenda on.wsj.com/...,http://on.wsj.com/ty6fGV,BEIJING—China plans to push for more funding f...,beijing—china plans push funding clean-energy ...
2,139191305766055904,139191305766055904,2011-11-22,PCCW Prices Telecom Trust IPO at Low End on....,http://on.wsj.com/sP81tk,HONG KONG—Fixed-line telecom operator PCCW Ltd...,hong kong—fixed-line telecom operator pccw ltd...
3,138978095397998592,138978095397998592,2011-11-22,Solar Companies Swing to Loss on.wsj.com/sH5...,http://on.wsj.com/sH5ICa,Chinese solar-products companies JA Solar Hold...,chinese solar-products companies ja solar hold...
4,138931945504636896,138931945504636896,2011-11-22,China Doubles Size of Hong Kong Currency Swa...,http://on.wsj.com/uza2y6,SHANGHAI—China and Hong Kong said Tuesday they...,shanghai—china hong kong said tuesday doubled ...
6,139309839816990704,139309839816990704,2011-11-23,Ma's Lead Narrows as Rivals Enter Taiwan Pol...,http://on.wsj.com/sXzmti,TAIPEI—Taiwan President Ma Ying-jeou is slippi...,taipei—taiwan president ying-jeou slipping pol...


In [9]:
nlp = en_core_web_sm.load()

In [10]:
def identify_label(text, label_type: List):
    label_text = []
    for x in nlp(text).ents:
        if x.label_ in label_type:
            label_text.append(x.text)
    return label_text

In [13]:
ORGS = []
for i in tqdm(range(len(news_df))):
    companies = identify_label(news_df.iloc[i]["clean_news_text"], label_type=["ORG"])
    ORGS.append(companies)

100%|██████████████████████████████████████████████████████████████████████████████| 2316/2316 [02:46<00:00, 13.94it/s]


In [26]:
news_df["companies_identified"] = ORGS

In [28]:
news_df.head()

,data-item-id,data-conversation-id,date,tweet,url,text,clean_news_text,companies_identified
0,138783141312741408,138783141312741408,2011-11-21,Chinese Media Firm Hit by Muddy Waters on.ws...,http://on.wsj.com/vMZ1s2,Focus Media Holding Ltd. lashed back Tuesday a...,focus media holding ltd. lashed back tuesday m...,"[focus media holding ltd, focus media, wsj mem..."
1,138950373275738096,138950373275738096,2011-11-22,China Pushes Clean-Energy Agenda on.wsj.com/...,http://on.wsj.com/ty6fGV,BEIJING—China plans to push for more funding f...,beijing—china plans push funding clean-energy ...,[dow jones]
2,139191305766055904,139191305766055904,2011-11-22,PCCW Prices Telecom Trust IPO at Low End on....,http://on.wsj.com/sP81tk,HONG KONG—Fixed-line telecom operator PCCW Ltd...,hong kong—fixed-line telecom operator pccw ltd...,"[telecom, pccw ltd, telecom trust, dow jones]"
3,138978095397998592,138978095397998592,2011-11-22,Solar Companies Swing to Loss on.wsj.com/sH5...,http://on.wsj.com/sH5ICa,Chinese solar-products companies JA Solar Hold...,chinese solar-products companies ja solar hold...,"[solar holdings co., suntech power holdings co..."
4,138931945504636896,138931945504636896,2011-11-22,China Doubles Size of Hong Kong Currency Swa...,http://on.wsj.com/uza2y6,SHANGHAI—China and Hong Kong said Tuesday they...,shanghai—china hong kong said tuesday doubled ...,"[hong kong monetary authority, de facto centra..."


In [45]:
news_df.to_excel(f"{EXCEL_PATH}\\wsj_logistics_news_companies_identified_{from_year}-{to_year}.xlsx",
                 index=False)

# Identifying url within text

In [80]:
urls = news_df["text"].apply(lambda x: find_urls_in_text(x) if (isinstance(x, str) and len(x) > 0) else []).values